In [1]:
import requests
import json
from random import randint
from tqdm import trange
from openai import OpenAI
import os
import pandas as pd
import pyarrow.parquet as pq
import re

In [3]:
def get_output(message):
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": message}
        ],
        temperature=0
    )
    response_message=json.loads(response.model_dump_json())
    return response_message['choices'][0]['message']['content']

In [4]:
def get_prompt(fact):
    template="""Please help me classify the following economic phenomenon.

Economic Phenomenon:
%s
"""%(fact)
    return template

In [5]:
def recognize_bias(bias):
    template='''The following is a financial task prompt may contain cognitive biases. Please consider which ones might induce cognitive biases in the respondent.

Example:[Your previous choice was contractionary economic phenomenon. This sentence contains an anchoring effect because it has presupposed an contractionary economic phenomenon.
At some point, continued large-scale trade deficits could trigger equilibrating, and possibly dislocating, changes in prices, interest rates, and exchange rates. This is a normal statement of economic phenomena, so it does not contain cognitive bias.]

Prompt: [%s]

Output Format: 
Bias sentence1:
...'''%(bias)
    return template

In [6]:
def rewrite_bias(bias,judge):
    template='''The following is a financial task may contain cognitive biases. Rewrite the task in [] such that a reviewer is not biased, while retaining the normal task.
Don't need to add option.

Task: [%s]

Here is new task without bias:
Task:'''%(bias)
    return template

In [7]:
def judge_bias(input):
    ans=get_output('''Please determine the follow prompt whether contain cognitive biases.

Example:
[ Your previous choice was A. This sentence contains cognitive biases because it gives a option.]
[ At some point, continued large-scale trade deficits could trigger equilibrating, and possibly dislocating, changes in prices, interest rates, and exchange rates. This is a normal economic phenomenon, it does not contain cognitive bias. ]

Prompt:[ %s ]

Output Format:
Contain or Not Contain'''%(input))
    #print('判断'+ans+'\n\n\n\n')
    if 'Not Contain' in ans or 'Not contain' in ans:
        return 0
    return 1

In [8]:
def judge_com(input):
    ans=get_output('''Please help me determine whether the following question is complete or not.

Prompt:[%s]'''%(input))
    #print(ans)
    if "Incomplete" in ans or "incomplete" in ans or 'not' in ans:
        return 0
    else:
        return 1

In [9]:
data= open('../data/test.jsonl','r')
for i in trange(100):
    question=json.loads(data.readline())
    linshi=get_prompt(question['question'])
    all_num=0
    while all_num<=0:
        prompt=linshi
        try_num=0
        while judge_bias(prompt) and try_num<=2:
            try:
                prompt=get_output(rewrite_bias(prompt))
                try:
                    prompt=re.findall(r"Task:([\S|\s]*)",prompt, re.S|re.I)[0]
                    prompt=prompt.replace('[','').replace(']','').replace('*','').strip()
                except:
                    a=1
                try_num=try_num+1 
            except:
                continue
        all_num=all_num+1
        if judge_com(prompt+'\n\nA.Expansionary economic phenomenon   B.Contractionary economic phenomenon'):
            break
    try:
        prompt=re.findall(r"Task:([\S|\s]*)",prompt, re.S|re.I)[0]
    except:
        a=1
    prompt=prompt.replace('[','').replace(']','').replace('*','').strip()
    ans=get_output(prompt+'\nA.Expansionary economic phenomenon   B.Contractionary economic phenomenon\n\nOutput format:\nAns: (The category of your choice)')
    with open("without_think.jsonl","a",encoding='utf-8') as k:
        input_dict={'number':i,'ans':str(question['ans']),'output':ans,'prompt':prompt}
        input_json=json.dumps(input_dict)
        k.write(input_json+'\n')

100%|██████████| 100/100 [23:14<00:00, 13.95s/it]  
